#### I am crating this notebook since I realised that I should consolidate all the preprocessing of the data before training

In [1]:
import pandas as pd
import numpy as np
import os
import re
from ast import literal_eval

In [2]:
df = pd.read_csv('/Users/louishagenbucher/Documents/GitHub/mtg_nlp_price_prediction/data/cards_data.csv')
df.sample(5)

,id,name,cmc,mana_cost,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,textless,rarity,reserved,released_at,booster,usd
11184,71ff86e9-1ce6-43cc-8135-f2adb1b6c5a7,In the Trenches,3.0,{1}{W}{W},['W'],False,['Enchantment'],['none'],Creatures you control get +1/+1.\n{5}{W}: Exil...,['no keywords'],non-creature,non-creature,False,mythic,False,2022-11-18,True,0.34
24643,b71b3df3-e188-45bb-8dc8-53fd4d527735,Vivid Marsh,0.0,{0},['No Colors'],False,['Land'],['none'],Vivid Marsh enters the battlefield tapped with...,['no keywords'],non-creature,non-creature,False,uncommon,False,2017-08-25,False,0.57
5742,b4fb87ab-8595-459a-a5f2-087296d9b120,Dissolve,3.0,{1}{U}{U},['U'],False,['Instant'],['none'],Counter target spell. Scry 1. (Look at the top...,['Scry'],non-creature,non-creature,False,common,False,2017-11-17,True,0.14
9793,a4276d10-d163-4068-a6db-5bb0a1ac88e3,Guttersnipe,3.0,{2}{R},['R'],False,['Creature'],"['Goblin', 'Shaman']","Whenever you cast an instant or sorcery spell,...",['no keywords'],2,2,False,common,False,2023-08-04,True,0.19
7319,65eb5920-3b03-4300-bc77-0fba5e6abe69,Faith's Shield,1.0,{W},['W'],False,['Instant'],['none'],Target permanent you control gains protection ...,['Fateful hour'],non-creature,non-creature,False,uncommon,False,2012-02-03,True,0.16


In [3]:
df.drop(columns=['id', 'mana_cost', 'textless'], inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26052 entries, 0 to 26051
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         26052 non-null  object 
 1   cmc          26052 non-null  float64
 2   colors       26052 non-null  object 
 3   legendary    26052 non-null  bool   
 4   types        26052 non-null  object 
 5   sub_types    26052 non-null  object 
 6   oracle_text  25099 non-null  object 
 7   keywords     26052 non-null  object 
 8   power        26052 non-null  object 
 9   toughness    26052 non-null  object 
 10  rarity       26052 non-null  object 
 11  reserved     26052 non-null  bool   
 12  released_at  26052 non-null  object 
 13  booster      26052 non-null  bool   
 14  usd          25996 non-null  float64
dtypes: bool(3), float64(2), object(10)
memory usage: 2.5+ MB


In [5]:
df['oracle_text'].fillna('notext', inplace=True)

In [6]:
df['sub_types'].fillna("['none']", inplace=True)

In [7]:
df['types'] = df['types'].apply(literal_eval)
df['sub_types'] = df['sub_types'].apply(literal_eval)
df['keywords'] = df['keywords'].apply(literal_eval)

In [8]:
df.drop(df[df['colors']=='[nan]'].index, inplace=True)
df.reset_index(drop=True)
df.head()

,name,cmc,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,rarity,reserved,released_at,booster,usd
0,+2 Mace,2.0,['W'],False,[Artifact],[Equipment],Equipped creature gets +2/+2.\nEquip {3} ({3}:...,[Equip],non-creature,non-creature,common,False,2021-07-23,True,0.02
1,Aarakocra Sneak,4.0,['U'],False,[Creature],"[Bird, Rogue]",Flying\nWhen Aarakocra Sneak enters the battle...,[Flying],1,4,common,False,2022-06-10,True,0.06
2,Abaddon the Despoiler,5.0,"['B', 'R', 'U']",True,[Creature],"[Astartes, Warrior]",Trample\nMark of Chaos Ascendant — During your...,"[Mark of Chaos Ascendant, Trample]",5,5,mythic,False,2022-10-07,False,2.81
3,Abandoned Outpost,0.0,['No Colors'],False,[Land],[none],Abandoned Outpost enters the battlefield tappe...,[no keywords],non-creature,non-creature,common,False,2001-10-01,True,0.14
4,Abandoned Sarcophagus,3.0,['C'],False,[Artifact],[none],You may cast spells that have a cycling abilit...,[no keywords],non-creature,non-creature,rare,False,2020-04-17,False,0.09


In [9]:
df['colors'].replace("['No Colors']", "['N']", inplace=True)
df['colors']=df['colors'].apply(literal_eval)
df.reset_index(drop=True, inplace=True)

In [10]:
df['power'] = df['power'].replace('non-creature', 0)
df['toughness'] = df['toughness'].replace('non-creature', 0)

In [11]:
df.drop(df[df['types'].apply(lambda x: '//' in x)].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [12]:
df['power'] = pd.to_numeric(df['power'], errors='coerce')
df['toughness'] = pd.to_numeric(df['toughness'], errors='coerce')
df['power'] = df['power'].fillna(-1).astype(int)
df['toughness'] = df['toughness'].fillna(-1).astype(int)

In [13]:
df['released_at'] = pd.to_datetime(df['released_at'])
reference_date = df['released_at'].min()
df['released_at'] = (df['released_at'] - reference_date).dt.days

In [14]:
df['cmc'] = df['cmc'].astype(int)

In [15]:
df.drop(columns='name', inplace=True)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25545 entries, 0 to 25544
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   cmc          25545 non-null  int64  
 1   colors       25545 non-null  object 
 2   legendary    25545 non-null  bool   
 3   types        25545 non-null  object 
 4   sub_types    25545 non-null  object 
 5   oracle_text  25545 non-null  object 
 6   keywords     25545 non-null  object 
 7   power        25545 non-null  int64  
 8   toughness    25545 non-null  int64  
 9   rarity       25545 non-null  object 
 10  reserved     25545 non-null  bool   
 11  released_at  25545 non-null  int64  
 12  booster      25545 non-null  bool   
 13  usd          25489 non-null  float64
dtypes: bool(3), float64(1), int64(4), object(6)
memory usage: 2.2+ MB


In [17]:
directory_path = '/Users/louishagenbucher/Documents/GitHub/mtg_nlp_price_prediction/data'
df.to_csv(os.path.join(directory_path, 'clean_cards.csv'), index=False)